In [1]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
import tensorflow as tf

In [2]:
# Parameters
img_width, img_height = 224, 224
batch_size = 4
epochs = 10
num_classes = 5
validation_split = 0.2  # 20% of the data will be used for validation
test_split = 0.1
print("Tets")
data_dir = './../datasets/dataset1'  # Update this to your dataset directory


# Data Generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=validation_split + test_split
)

# Augmentation parameters for specific classes
specific_class_augmentation = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=validation_split + test_split
)

# Generators for training and validation
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=validation_split / (validation_split + test_split)
)

validation_generator = validation_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Data preparation for testing
test_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=test_split / (validation_split + test_split)
)

test_generator = test_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


# Oversampling the specified minority classes with augmentation
specific_classes = ['1', '2', '3', '4']  # Example specific class indices as strings
oversample_ratio = 2  # How many times to oversample the specific classes

# Create separate generators for the specific classes with augmentation
specific_class_generators = []
for cls in specific_classes:
    specific_class_generator = specific_class_augmentation.flow_from_directory(
        data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        classes=[cls],
        subset='training',
        shuffle=True
    )
    specific_class_generators.append(specific_class_generator)

# Function to combine generators
def combined_generator(base_generator, specific_class_generators, oversample_ratio):
    while True:
        x_batch, y_batch = base_generator.next()
        for _ in range(oversample_ratio):
            specific_class_index = np.random.randint(len(specific_class_generators))
            x_specific, y_specific = specific_class_generators[specific_class_index].next()
            
            # Ensure y_specific matches the shape of y_batch
            if y_specific.shape[1] == 1:
                y_specific = np.eye(num_classes)[y_specific[:, 0].astype(int)]  # One-hot encode if necessary
                
            x_batch = np.concatenate((x_batch, x_specific), axis=0)
            y_batch = np.concatenate((y_batch, y_specific), axis=0)
        
        yield x_batch, y_batch

# Combined generator for training
combined_train_generator = combined_generator(train_generator, specific_class_generators, oversample_ratio)

class_weights = compute_class_weight('balanced', classes=np.unique(train_generator.classes), y=train_generator.classes)
class_weights_dict = dict(enumerate(class_weights))

print("Class weights: ", class_weights_dict)


Tets
Found 24591 images belonging to 5 classes.
Found 23416 images belonging to 5 classes.
Found 11708 images belonging to 5 classes.
Found 1711 images belonging to 1 classes.
Found 3705 images belonging to 1 classes.
Found 612 images belonging to 1 classes.
Found 496 images belonging to 1 classes.
Class weights:  {0: 0.2722200697404107, 1: 2.874459380479252, 2: 1.3274493927125506, 3: 8.036274509803922, 4: 9.915725806451613}


In [3]:
# Define a custom weighted categorical cross-entropy loss function
def weighted_categorical_crossentropy(weights):
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        weights_tensor = tf.reduce_sum(weights * y_true, axis=-1)
        unweighted_losses = categorical_crossentropy(y_true, y_pred)
        weighted_losses = unweighted_losses * weights_tensor
        return tf.reduce_mean(weighted_losses)
    return loss

# Define the custom loss function using the computed class weights
loss_fn = weighted_categorical_crossentropy(class_weights)

In [4]:
from sklearn.metrics import balanced_accuracy_score, precision_recall_fscore_support


def balanced_accuracy(y_true, y_pred):
    y_true = tf.argmax(y_true, axis=1)
    y_pred = tf.argmax(y_pred, axis=1)
    return tf.py_function(
        lambda y_true, y_pred: balanced_accuracy_score(
            y_true.numpy(), y_pred.numpy()),
        (y_true, y_pred),
        tf.float64)


def fscore(y_true, y_pred):
    y_true = tf.cast(tf.argmax(y_true, axis=1), tf.int32)
    y_pred = tf.cast(tf.argmax(y_pred, axis=1), tf.int32)

    def compute_fscore(y_true, y_pred):
        _, _, fscore, _ = precision_recall_fscore_support(
            y_true, y_pred, average='macro', zero_division=0)
        return fscore

    fscore = tf.py_function(
        compute_fscore, (y_true, y_pred), tf.float64)
    return fscore

In [5]:
epochs = 10
learning_rate = 3.9e-5
metrics = [
    'accuracy',
    tf.keras.metrics.CategoricalAccuracy(),
    tf.keras.metrics.AUC(),
    balanced_accuracy,
    fscore,
    tf.keras.metrics.Precision(),
    tf.keras.metrics.Recall()
]

In [6]:
import warnings
warnings.filterwarnings('ignore', message="A single label was found in y_true and y_pred.")
warnings.filterwarnings('ignore', message="y_pred contains classes not in y_true") 
warnings.filterwarnings('ignore', message="A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.") 


In [10]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
# Load the InceptionV3 model, excluding the top layer

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

In [13]:


# Compile the model with the custom loss function
model.compile(optimizer=Adam(learning_rate=learning_rate), loss=loss_fn, metrics=metrics)

# Train the model
history = model.fit(
    combined_train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    epochs=epochs
)

Epoch 1/10
6148/6148 [==============================] - 644s 103ms/step - loss: 0.7731 - accuracy: 0.7203 - categorical_accuracy: 0.7203 - auc: 0.9425 - balanced_accuracy: 0.4657 - fscore: 0.3832 - precision: 0.8038 - recall: 0.6780 - val_loss: 1.7625 - val_accuracy: 0.2358 - val_categorical_accuracy: 0.2358 - val_auc: 0.6206 - val_balanced_accuracy: 0.2353 - val_fscore: 0.1637 - val_precision: 0.1578 - val_recall: 0.0800
Epoch 2/10
6148/6148 [==============================] - 632s 103ms/step - loss: 0.6459 - accuracy: 0.7454 - categorical_accuracy: 0.7454 - auc: 0.9541 - balanced_accuracy: 0.5166 - fscore: 0.4271 - precision: 0.8474 - recall: 0.6860 - val_loss: 1.6302 - val_accuracy: 0.2941 - val_categorical_accuracy: 0.2941 - val_auc: 0.7034 - val_balanced_accuracy: 0.3089 - val_fscore: 0.2090 - val_precision: 0.2425 - val_recall: 0.0842
Epoch 3/10
6148/6148 [==============================] - 617s 100ms/step - loss: 0.6030 - accuracy: 0.7559 - categorical_accuracy: 0.7559 - auc: 0.95

In [19]:
# Compute confusion matrix and classification report
from sklearn.metrics import classification_report, confusion_matrix

# Compute confusion matrix and classification report
y_true = test_generator.classes
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

print(classification_report(y_true, y_pred_classes, target_names=list(test_generator.class_indices.keys())))
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print(conf_matrix)


2927/2927 [==============================] - 44s 15ms/step
              precision    recall  f1-score   support

           0       0.73      0.13      0.22      8603
           1       0.07      0.46      0.12       814
           2       0.14      0.29      0.19      1764
           3       0.03      0.06      0.04       291
           4       0.04      0.08      0.05       236

    accuracy                           0.17     11708
   macro avg       0.20      0.20      0.12     11708
weighted avg       0.57      0.17      0.20     11708

[[1098 4146 2576  436  347]
 [ 114  377  240   49   34]
 [ 221  855  506  104   78]
 [  38  121  100   18   14]
 [  26  111   70   11   18]]


In [20]:
import matplotlib.pyplot as plt
import seaborn as sns


# Function to plot confusion matrix
def plot_confusion_matrix(cm, class_names):
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.title('Confusion Matrix')
    plt.show()

# Plot the confusion matrix
plot_confusion_matrix(conf_matrix, list(test_generator.class_indices.keys())

SyntaxError: incomplete input (2527156713.py, line 15)

In [11]:
for layer in base_model.layers[:249]:  # Adjust the index as needed
    layer.trainable = False
for layer in base_model.layers[249:]:
    layer.trainable = True

# Recompile the model with a lower learning rate
# model.compile(optimizer=Adam(lr=0.00001), loss=loss_fn, metrics=['accuracy'])
model.compile(optimizer=Adam(lr=learning_rate), loss="categorical_crossentropy", metrics=metrics)

# Continue training (fine-tuning)
# Train the model
history = model.fit(
    combined_train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    epochs=epochs
)

Epoch 1/10


c:\Users\Computing\anaconda3\envs\RML\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


6148/6148 [==============================] - 637s 102ms/step - loss: 0.2849 - accuracy: 0.9065 - categorical_accuracy: 0.9065 - auc: 0.9890 - balanced_accuracy: 0.7806 - fscore: 0.7252 - precision: 0.9217 - recall: 0.8929 - val_loss: 0.7302 - val_accuracy: 0.7518 - val_categorical_accuracy: 0.7518 - val_auc: 0.9293 - val_balanced_accuracy: 0.6414 - val_fscore: 0.5702 - val_precision: 0.7814 - val_recall: 0.7168
Epoch 2/10
6148/6148 [==============================] - 644s 105ms/step - loss: 0.2500 - accuracy: 0.9148 - categorical_accuracy: 0.9148 - auc: 0.9915 - balanced_accuracy: 0.8022 - fscore: 0.7416 - precision: 0.9299 - recall: 0.9023 - val_loss: 0.7201 - val_accuracy: 0.7528 - val_categorical_accuracy: 0.7528 - val_auc: 0.9323 - val_balanced_accuracy: 0.6634 - val_fscore: 0.5841 - val_precision: 0.7996 - val_recall: 0.6879
Epoch 3/10
6148/6148 [==============================] - 624s 101ms/step - loss: 0.2288 - accuracy: 0.9204 - categorical_accuracy: 0.9204 - auc: 0.9929 - balanc